In [1]:
"""
    Name: Jacqueline Ramos
    Description: Below is my program for the Mindex Data & Analytics Code Challenge. 
"""

'\n    Name: Jacqueline Ramos\n    Description: Below is my program for the Mindex Data & Analytics Code Challenge. \n'

In [2]:
import pandas as pd
import boto3 

In [3]:
# create an AWS client to connect to AWS S3 bucket
client=boto3.client('s3')

# list of files to download from bucket
file_list=["bengals.csv", "boyd_receiving.csv", "chase_receiving.csv", "higgins_receiving.csv"]

# function downloads each file from 'mindex-data-analytics-code-challenge' bucket to my local folder
def download_files(files:list):
    for file in files:
        client.download_file(Bucket="mindex-data-analytics-code-challenge",
        Key=file,
        Filename="/Users/jacquelineramos/Documents/MindexChallenge/"+file)

download_files(file_list)


In [4]:
# read each csv file into a pandas dataframe
bengals_df = pd.read_csv('bengals.csv', sep=',')
boyd_df = pd.read_csv('boyd_receiving.csv', sep=',')
chase_df = pd.read_csv('chase_receiving.csv', sep=',')
higgins_df = pd.read_csv('higgins_receiving.csv', sep=',')

In [8]:
# rename Yards and TD column names to include receiver's name 
boyd_df.rename(columns = {'Yards': 'Boyd_Yards', 'TD': 'Boyd_TD'}, inplace=True)

chase_df.rename(columns = {'Yards': 'Chase_Yards', 'TD': 'Chase_TD'}, inplace=True)

higgins_df.rename(columns = {'Yards': 'Higgins_Yards', 'TD': 'Higgins_TD'}, inplace=True)


In [10]:
# merge the dataframes together to contain all 3 different receivers yards and TD stats 
# along with the bengals week, opponent, location, and result stats
merged_df=pd.merge(bengals_df, boyd_df, how='left').merge(chase_df, how='left').merge(higgins_df, how='left')


In [11]:
# replace the 1.0 and 0.0 values with Win and Loss in the Result column
merged_df['Result']=merged_df['Result'].replace({1.0: 'Win', 0.0: 'Loss'})
print(merged_df)

     Week Opponent Location Result  Boyd_Yards  Boyd_TD  Chase_Yards  \
0    PRE1       TB     Away    Win         NaN      NaN          NaN   
1    PRE2      WSH     Away   Loss         NaN      NaN          NaN   
2    PRE3      MIA     Home   Loss         NaN      NaN          NaN   
3    REG1      MIN     Home    Win        32.0      0.0        101.0   
4    REG2      CHI     Away   Loss        73.0      0.0         54.0   
5    REG3      PIT     Away    Win        36.0      1.0         65.0   
6    REG4      JAX     Home    Win       118.0      0.0         77.0   
7    REG5       GB     Home   Loss        24.0      0.0        159.0   
8    REG6      DET     Away    Win         7.0      0.0         97.0   
9    REG7      BAL     Away    Win        39.0      0.0        201.0   
10   REG8      NYJ     Away   Loss        69.0      1.0         32.0   
11   REG9      CLE     Home   Loss        11.0      0.0         49.0   
12  REG10      NaN      NaN    NaN         NaN      NaN         